In [ ]:
from datetime import datetime
import speech_recognition as sr
import pyttsx3
import webbrowser
import wikipedia
import wolframalpha
import openai
import os
from dotenv import load_dotenv
import google.cloud.texttospeech as tts
import pygame
import time
import math
import requests
import json

load_dotenv()

# Mute ALSA errors
from ctypes import *
from contextlib import contextmanager

ERROR_HANDLER_FUNC = CFUNCTYPE(None, c_char_p, c_int, c_char_p, c_int, c_char_p)

def py_error_handler(filename, line, function, err, fmt):
    pass

c_error_handler = ERROR_HANDLER_FUNC(py_error_handler)

@contextmanager
def noalsaerr():
    try:
        asound = cdll.LoadLibrary('libasound.so')
        asound.snd_lib_error_set_handler(c_error_handler)
        yield
        asound.snd_lib_error_set_handler(None)
    except:
        yield
        print('')

### PARAMETERS ###
tts_type = 'local'  # google or local

r = sr.Recognizer()

engine = pyttsx3.init('sapi5')
voices = engine.getProperty('voices')
# Change the voice to sound more like JARVIS
jarvis_voice_id = voices[1].id
engine.setProperty('voice', jarvis_voice_id)

def speak(text, rate=120):
    print(f"Speaking: {text}")  # Debug print
    time.sleep(0.3)
    try:
        if tts_type == 'local':
            engine.setProperty('rate', rate)
            engine.say(text)
            engine.runAndWait()
        if tts_type == 'google':
            speech = google_text_to_wav('en-US-News-K', text)
            pygame.mixer.init(frequency=12000, buffer=512)
            speech_sound = pygame.mixer.Sound(speech)
            speech_length = int(math.ceil(pygame.mixer.Sound.get_length(speech_sound)))
            speech_sound.play()
            time.sleep(speech_length)
            pygame.mixer.quit()
    except KeyboardInterrupt:
        try:
            if tts_type == 'google':
                pygame.mixer.quit()
        except:
            pass
        return

def take_command():
    with sr.Microphone() as source:
        print("Listening.....")
        r.pause_threshold = 1
        audio = r.listen(source)
    try:
        print("Wait for a few moments")
        query = r.recognize_google(audio, language="en-in")
        print("User said ", query)
        return query.lower()
    except Exception as e:
        print(e)
        speak("Say that again, please")
        return None

def wish_me():
    hour = int(datetime.now().hour)
    if hour >= 0 and hour < 12:
        speak("Good Morning!")
    elif hour >= 12 and hour < 18:
        speak("Good Afternoon!")
    else:
        speak("Good Evening!")

def google_text_to_wav(voice_name: str, text: str):
    language_code = "-".join(voice_name.split("-")[:2])

    text_input = tts.SynthesisInput(text=text)
    voice_params = tts.VoiceSelectionParams(language_code=language_code, name=voice_name)
    audio_config = tts.AudioConfig(audio_encoding=tts.AudioEncoding.LINEAR16)

    client = tts.TextToSpeechClient()
    response = client.synthesize_speech(input=text_input, voice=voice_params, audio_config=audio_config)

    return response.audio_content

chrome_path=r"C:\Program Files\Google\Chrome\Application\chrome.exe"
webbrowser.register('chrome',None,webbrowser.BackgroundBrowser(chrome_path))

appId='A767Q9-WGP8QUVJJ8'
wolframClient=wolframalpha.Client(appId)

def parse_command():
    with noalsaerr():
        listener = sr.Recognizer()
        print('Listening for a command')

        with sr.Microphone() as source:
            listener.pause_threshold = 2
            input_speech = listener.listen(source)

        try:
            print('Recognizing speech...')
            query = listener.recognize_google(input_speech, language='en_gb')
            print(f'The input speech was: {query}')

            # Check if the query contains arithmetic operations
            if any(op in query for op in ['+', '-', '*', '/']):
                return query  # Return the arithmetic query directly

            # Handle other types of queries here if needed

        except Exception as exception:
            print('I did not quite catch that')
            print(exception)
            return 'None'
        return query.lower()

def search_wikipedia(keyword=''):
    try:
        search_results = wikipedia.search(keyword)
        if not search_results:
            return 'No result received'
        
        # Try the first few search results to find a valid page
        for result in search_results[:3]:
            try:
                wiki_page = wikipedia.page(result)
                return wiki_page.summary
            except wikipedia.DisambiguationError as e:
                print(f"Disambiguation error: {e}. Trying next result.")
                continue
            except wikipedia.PageError as e:
                print(f"Page error: {e}. Trying next result.")
                continue
        
        # If no valid page is found
        return 'Page not found'
    
    except Exception as e:
        print(f"An error occurred while searching Wikipedia: {e}")
        return 'An error occurred while searching Wikipedia.'

def list_or_dict(var):
    if isinstance(var, list):
        return var[0]['plaintext']
    else:
        return var['plaintext']

def search_wolframalpha(keyword=''):
    response = wolframClient.query(keyword)
    if response['@success'] == 'false':
        return 'I could not compute that.'
    else:
        result = ''
        pod0 = response['pod'][0]
        pod1 = response['pod'][1]
        if (('result') in pod1['@title'].lower()) or (pod1.get('@primary', 'false') == 'true') or ('definition' in pod1['@title'].lower()):
            result = list_or_dict(pod1['subpod'])
            return result.split('(')[0]
        else:
            question = list_or_dict(pod0['subpod'])
            question = question.split('(')[0]
            return question

def query_openai(prompt=""):
    openai.organization = os.environ['OPENAI_ORG']
    openai.api_key = os.environ['OPENAI_API_KEY']
    try:
        response = openai.Completion.create(engine="text-davinci-003", prompt=prompt, temperature=0.3, max_tokens=80)
        print(f"OpenAI response: {response}")  # Debug print
        return response.choices[0].text.strip()
    except Exception as e:
        print(f"Error querying OpenAI: {e}")  # Debug print
        return "Sorry, I could not process that request."

def open_website(url):
    url_mapping = {
        "google maps": "https://maps.google.com",
        "youtube": "https://www.youtube.com",
        "google": "https://www.google.com",
        "facebook": "https://www.facebook.com",
        # Add more predefined mappings as needed
    }

    if url in url_mapping:
        full_url = url_mapping[url]
    else:
        if not url.startswith('http'):
            url = 'https://' + url + '.com'
        full_url = url

    try:
        webbrowser.open(full_url)
        speak(f"Opening {url}")
    except Exception as e:
        print(e)
        speak("Sorry, I am unable to open the website.")

def get_time():
    now = datetime.now().strftime("%H:%M:%S")
    speak(f"The current time is {now}")

def get_date():
    today = datetime.now().strftime("%Y-%m-%d")
    speak(f"Today's date is {today}")

def get_weather(city):
    api_key = os.getenv('48341adc40a5bfe882b62ee145bd1a54')
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = base_url + "appid=" + api_key + "&q=" + city
    response = requests.get(complete_url)
    data = response.json()
    if data["cod"] != "404":
        main = data["main"]
        temperature = main["temp"]
        pressure = main["pressure"]
        humidity = main["humidity"]
        weather_desc = data["weather"][0]["description"]
        temp_celsius = temperature - 273.15
        speak(f"The temperature in {city} is {temp_celsius:.2f} degrees Celsius with {weather_desc}. The atmospheric pressure is {pressure} hPa and humidity is {humidity}%.")
    else:
        speak("City not found")

def get_news():
    api_key = os.getenv('00134e89d6f04190ac3b4116482e7f77')
    url = f"https://newsapi.org/v2/top-headlines?country=us&apiKey={api_key}"
    response = requests.get(url)
    news_data = response.json()
    articles = news_data["articles"]
    top_five_news = articles[:5]
    for i, article in enumerate(top_five_news, start=1):4
    speak(f"News {i}: {article['title']}")

def get_stock_price(stock_symbol):
    api_key = os.getenv('STOCK_API_KEY5LLIS55CVET8TPI1')
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=RELIANCE.BSE&outputsize=full&apikey=demo"
    response = requests.get(url)
    data = response.json()
    if "Time Series (5min)" in data:
        time_series = data["Time Series (5min)"]
        latest_timestamp = list(time_series.keys())[0]
        latest_data = time_series[latest_timestamp]
        closing_price = latest_data["4. close"]
        speak(f"The latest closing price of {stock_symbol} is {closing_price} dollars.")
    else:
        speak("Stock data not found")

def write_code(language, task):
    openai.organization = os.environ['OPENAI_ORG']
    openai.api_key = os.environ['OPENAI_API_KEY']
    prompt = f"Write a {language} code to {task}."
    try:
        response = openai.Completion.create(engine="code-davinci-002", prompt=prompt, temperature=0.5, max_tokens=200)
        code = response.choices[0].text.strip()
        return code
    except Exception as e:
        print(f"Error generating code: {e}")
        return "Sorry, I couldn't generate the code."

if __name__ == '__main__':
    wish_me()
    while True:
        query = parse_command()
        if query:
            if 'how are you' in query:
                speak("I am just a program, so I don't have feelings, but thank you for asking!")
            elif 'open' in query:
                speak("Opening the website")
                url = query.replace('open', '').strip()
                open_website(url)
            elif 'wikipedia' in query:
                speak("Searching Wikipedia")
                query = query.replace("wikipedia", "").strip()
                results = search_wikipedia(query)
                speak("According to Wikipedia")
                speak(results)
                print(results)
            elif 'compute' in query or 'calculate' in query:
                query = query.replace('compute', '').replace('calculate', '').strip()
                try:
                    result = search_wolframalpha(query)
                    speak(result)
                except:
                    speak('Unable to compute')
            elif 'log' in query:
                speak('Ready to record your note')
                new_note = parse_command().lower()
                now = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
                with open(f'note_{now}.txt', 'w') as newFile:
                    newFile.write(now)
                    newFile.write(' ')
                    newFile.write(new_note)
                speak('Note written')
            elif 'ask chatgpt' in query:
                query = query.replace('ask chatgpt', '').strip()
                response = query_openai(query)
                speak(response)
            elif 'time' in query:
                get_time()
            elif 'date' in query:
                get_date()
            elif 'weather' in query:
                city = query.replace('weather', '').strip()
                get_weather(city)
            elif 'news' in query:
                get_news()
            elif 'stock price' in query:
                stock_symbol = query.replace('stock price', '').strip()
                get_stock_price(stock_symbol)
            elif 'write code' in query:
                speak('Which programming language?')
                language = parse_command().lower()
                speak(f'What do you want to do in {language}?')
                task = parse_command().lower()
                code = write_code(language, task)
                speak("Here's the code")
                print(code)
                speak(code)
            elif 'exit' in query:
                speak('Goodbye')
                break
            else:
                try:
                    result = search_wolframalpha(query)
                    speak(result)
                except:
                    try:
                        results = search_wikipedia(query)
                        speak("According to Wikipedia")
                        speak(results)
                        print(results)
                    except:
                        response = query_openai(query)
                        speak(response)


pygame 2.6.0 (SDL 2.28.4, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html
Speaking: Good Afternoon!
Listening for a command
Recognizing speech...
The input speech was: try



c:\Users\Mohith\AppData\Local\Programs\Python\Python310\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\Mohith\AppData\Local\Programs\Python\Python310\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Disambiguation error: "Try" may refer to: 
Try (Bebo Norman album)
Try!
"Try" (Blue Rodeo song)
"Try" (Colbie Caillat song)
"Try" (Michael Penn song)
"Try" (Nelly Furtado song)
"Try" (Pink song)
"Try" (Pseudo Echo song)
"Try" (Rick Astley song)
"Try" (Schiller song)
Try (Just a Little Bit Harder)
In a World Like This
Blue Smoke
Calumet (album)
Strip Me
Brutalism
Painless
About Time
The Magic Numbers
Coexist
Feeding off the Mojo
Room Service
Try (rugby)
conversion
"Try" (The Killing)
"Try" (The Walking Dead)
Turkish lira
Lily Burana
exception handling syntax
whaling
Try Bennett
Try Hamdani
Poa Try
Try Sutrisno
Try Chheang Huot
Kjetil Try
All pages with titles containing Try
Trie. Trying next result.
Speaking: According to Wikipedia
Speaking: "Try, Try, Try" is a song by American alternative rock band The Smashing Pumpkins. It was the third and final single from the band's fifth studio album, Machina/The Machines of God, and was released on September 11, 2000. It was written by Billy Cor